In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('winequality-white.csv', sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [5]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [6]:
# Separate the target feature, split data in 7:3 proportion
# (30% form a holdout set, use random_state=17), and preprocess
# data with StandardScaler.

# We split data into training and validation parts.
target_feature = data['quality']
other_features = data.drop('quality', axis=1)  # изымаем целевой признак

X_train, X_holdout, y_train, y_holdout = train_test_split(
    other_features,
    target_feature,
    test_size=0.3,
    random_state=17
)

# Идея StandardScaler заключается в том, что он преобразует
# (нормализует/стандартизирует) данные таким образом, что его 
# распределение будет иметь среднее значение(mean) 0 и стандартное отклонение 
# (standard deviation) 1. Учитывая распределение данных, каждое значение в 
# наборе данныхбудет вычтено из среднего значения выборки, а затем 
# разделено по стандартному отклонению всего набора данных.

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [7]:
# Train a simple linear regression model (Ordinary Least Squares).

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
prediction_holdout_linear = linreg.predict(X_holdout_scaled)
prediction_train_linear = linreg.predict(X_train_scaled)

In [8]:
# What are mean squared errors of model predictions on train and holdout sets?

print(f"""Mean squared error {mean_squared_error(
    y_train,
    prediction_train_linear
)}""")

print(f"""Mean squared error {mean_squared_error(
    y_holdout,
    prediction_holdout_linear
)}""")

Mean squared error 0.558060648980358
Mean squared error 0.584247310240454


In [9]:
# Sort features by their influence on the target feature (wine quality).
# Beware that both large positive and large negative coefficients mean
# large influence on target. It's handy to use pandas.DataFrame here.

# Which feature this linear regression model treats as the 
# most influential on wine quality?

# Еще одно отличие серий от списков — в качестве индексов можно использовать
# произвольные значения, это делает данные нагляднее.

linreg_coef = pd.DataFrame(
    {
        'coef': linreg.coef_,   # скорость изменения свойства
        'coef_abs': np.abs(linreg.coef_)  
    },
    index=data.columns.drop('quality')
)
linreg_coef.sort_values(
    by='coef_abs',
    ascending=False
)

,coef,coef_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


In [10]:
# Train a LASSO model with a=0.01 (weak regularization) and
# scaled data. Again, set random_state=17.

# LASSO(Least Absolute Shrinkage and Selection Operator), L1-регуляризация
# Штрафы в ней начисляются лишь за признаки с большим значением коэффициентов.
# К тому же лассо может обнулять значения коэффициентов, тем самым полностью 
# убирая признак из датасета (так как при вычислении результирующей переменной
# соответствующий признак будет умножен на ноль).


# Here, alpha is the parameter which balances the amount of emphasis given 
# to minimizing RSS(Residual Sum of Squares) vs minimizing sum of square of coefficients. Alpha can 
# take various values:
# Objective = RSS + alpha * (sum of square of coefficients)

# alpha = 0:
# The objective becomes same as simple linear regression.
# We’ll get the same coefficients as simple linear regression.

# alpha = ∞:
# The coefficients will be zero. Why? Because of infinite weightage on square
# of coefficients, anything less than zero will make the objective infinite.

# 0 < alpha < ∞:
# The magnitude of alpha will decide the weightage given to different parts of objective.
# The coefficients will be somewhere between 0 and ones for simple linear regression.


lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)
lasso_with_alpha_train = lasso1.predict(X_train_scaled)
lasso_with_alpha_test = lasso1.predict(X_holdout_scaled)

In [11]:
print(f"""Mean squared error {mean_squared_error(
    y_train,
    lasso_with_alpha_train
)}""")

print(f"""Mean squared error {mean_squared_error(
    y_holdout,
    lasso_with_alpha_test
)}""")

Mean squared error 0.5637869195669829
Mean squared error 0.5736627127525903


In [12]:
# Which feature is the least informative in predicting wine quality,
# according to this LASSO model?

lasso1_coef = pd.DataFrame(
    {
        'coef': lasso1.coef_,   # скорость изменения свойства
        'coef_abs': np.abs(lasso1.coef_)
    },
    index=data.columns.drop('quality')
)
lasso1_coef.sort_values(
    by='coef_abs',
    ascending=False
)


,coef,coef_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


In [13]:
# Train LassoCV with random_state=17 to choose 
# the best value of alpha in 5-fold cross-validation.

# cross-validation/перекрёстная проверка/кросс проверка/скользящий контроль —
# метод оценки аналитической модели и её поведения на независимых данных.
# При оценке модели имеющиеся в наличии данные разбиваются на k частей.
# Затем на k−1 частях данных производится обучение модели, а оставшаяся
# часть данных используется для тестирования. Процедура повторяется k раз;
# в итоге каждая из k частей данных используется для тестирования.
# В результате получается оценка эффективности выбранной модели с наиболее
# равномерным использованием имеющихся данных.

alphas = np.logspace(-6, 2, 200)

lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)
lasso_cv.alpha_

0.0002833096101839324

In [14]:
print(alphas)

[1.00000000e-06 1.09698580e-06 1.20337784e-06 1.32008840e-06
 1.44811823e-06 1.58856513e-06 1.74263339e-06 1.91164408e-06
 2.09704640e-06 2.30043012e-06 2.52353917e-06 2.76828663e-06
 3.03677112e-06 3.33129479e-06 3.65438307e-06 4.00880633e-06
 4.39760361e-06 4.82410870e-06 5.29197874e-06 5.80522552e-06
 6.36824994e-06 6.98587975e-06 7.66341087e-06 8.40665289e-06
 9.22197882e-06 1.01163798e-05 1.10975250e-05 1.21738273e-05
 1.33545156e-05 1.46497140e-05 1.60705282e-05 1.76291412e-05
 1.93389175e-05 2.12145178e-05 2.32720248e-05 2.55290807e-05
 2.80050389e-05 3.07211300e-05 3.37006433e-05 3.69691271e-05
 4.05546074e-05 4.44878283e-05 4.88025158e-05 5.35356668e-05
 5.87278661e-05 6.44236351e-05 7.06718127e-05 7.75259749e-05
 8.50448934e-05 9.32930403e-05 1.02341140e-04 1.12266777e-04
 1.23155060e-04 1.35099352e-04 1.48202071e-04 1.62575567e-04
 1.78343088e-04 1.95639834e-04 2.14614120e-04 2.35428641e-04
 2.58261876e-04 2.83309610e-04 3.10786619e-04 3.40928507e-04
 3.73993730e-04 4.102658

In [15]:
# Which feature is the least informative in predicting wine quality,
# according to the tuned LASSO model?

lasso_cv_coef = pd.DataFrame(
    {
        'coef': lasso_cv.coef_,  # скорость изменения свойства
        'coef_abs': np.abs(lasso_cv.coef_)
    },
    index=data.columns.drop('quality')
)
lasso_cv_coef.sort_values(
    by='coef_abs',
    ascending=False
)

,coef,coef_abs
density,-0.648161,0.648161
residual sugar,0.526883,0.526883
volatile acidity,-0.192049,0.192049
pH,0.146549,0.146549
alcohol,0.137115,0.137115
fixed acidity,0.093295,0.093295
sulphates,0.060939,0.060939
free sulfur dioxide,0.042698,0.042698
total sulfur dioxide,0.012969,0.012969
chlorides,0.006933,0.006933


In [27]:
print(lasso_cv_coef.index)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [16]:
# What are mean squared errors of tuned 
# LASSO predictions on train and holdout sets?
prediction_holdout_lasso = lasso_cv.predict(X_holdout_scaled)
prediction_train_lasso = lasso_cv.predict(X_train_scaled)

print(f"""Mean squared error {mean_squared_error(
    y_train,
    prediction_train_lasso
)}""")

print(f"""Mean squared error {mean_squared_error(
    y_holdout,
    prediction_holdout_lasso
)}""")

Mean squared error 0.5580700141873788
Mean squared error 0.5832976077860629


In [17]:
# Train a Random Forest with out-of-the-box parameters,
# setting only random_state to be 17.

# RF (random forest) — это множество решающих деревьев. В задаче регрессии
# их ответы усредняются, в задаче классификации принимается решение 
# голосованием по большинству.

# Все деревья строятся независимо по следующей схеме:
# Выбирается подвыборка обучающей выборки размера samplesize (м.б. с 
# возвращением) – по ней строится дерево (для каждого дерева — своя 
# подвыборка).
# Для построения каждого расщепления в дереве просматриваем max_features
# случайных признаков (для каждого нового расщепления — свои случайные 
# признаки).
# Выбирается наилучший признак и расщепление по нему (по заранее заданному 
# критерию). Дерево строится, как правило, до исчерпания выборки (пока в 
# листьях не останутся представители только одного класса), но в современных
# реализациях есть параметры, которые ограничивают высоту дерева, число 
# объектов в листьях и число объектов в подвыборке, при котором проводится расщепление.


forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

In [18]:
# What are mean squared errors of RF model on the training set, 
# in cross-validation (cross_val_score with scoring='neg_mean_squared_error'
# and other arguments left with default values) and on holdout set?

print(f"""Mean squared error {mean_squared_error(
    y_train,
    forest.predict(X_train_scaled)
)}""")

print(f"""Mean squared error {np.mean(
    np.abs(cross_val_score(
        forest,
        X_train_scaled,
        y_train,
        scoring='neg_mean_squared_error'
    )
))}""")

print(f"""Mean squared error {mean_squared_error(
    y_holdout,
    forest.predict(X_holdout_scaled)
)}""")

Mean squared error 0.05261155192532089
Mean squared error 0.4142003732204039
Mean squared error 0.37163775510204083


In [19]:
# Tune the max_features and max_depth hyper-parameters with GridSearchCV and
# again check mean cross-validation MSE and MSE on holdout set.

# Гиперпараметры
forest_params = {
    'max_depth': list(range(10, 25)),
    'max_features': list(range(6,12))
}

locally_best_forest = GridSearchCV(
    RandomForestRegressor(
        n_jobs=-1,
        random_state=17
    ),
    forest_params,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=5,
    verbose=True
)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  3.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-

In [20]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6}, -0.3977328819150594)

In [21]:
# What are mean squared errors of tuned RF model in cross-validation 
# (cross_val_score with scoring='neg_mean_squared_error' and other 
# arguments left with default values) and on holdout set?

print(f"""Mean squared error {np.mean(
    np.abs(cross_val_score(
        locally_best_forest.best_estimator_,
        X_train_scaled,
        y_train,
        scoring='neg_mean_squared_error'
    )
))}""")

print(f"""Mean squared error {mean_squared_error(
    y_holdout,
    locally_best_forest.predict(X_holdout_scaled)
)}""")

Mean squared error 0.39773288191505934
Mean squared error 0.36572455603132475


In [22]:
# What is the most important feature, according to the Random Forest model?

rf_importance = pd.DataFrame(
    locally_best_forest.best_estimator_.feature_importances_,
    columns=['coef'],
    index=data.columns[:-1]
) 
rf_importance.sort_values(by='coef', ascending=False)

,coef
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813


In [25]:
print(rf_importance.index)


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')
